In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.metrics import RootMeanSquaredError


def load_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data


def preprocess_data(data, lookback, days_ahead):
    data = data[['Close']]
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    X = []
    y = []
    for i in range(lookback, len(data) - days_ahead):
        X.append(scaled_data[i - lookback:i, 0])
        y.append(scaled_data[i + days_ahead, 0])

    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    return X, y, scaler


ticker = 'AAPL'
start_date = '2020-01-01'
end_date = '2021-12-31'
lookback = 120
days_ahead = 5

# Load historical stock data
data = load_data(ticker, start_date, end_date)

# Preprocess the data
X, y, scaler = preprocess_data(data, lookback, days_ahead)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(Bidirectional(LSTM(units=100, return_sequences=True, input_shape=(X_train.shape[1], 1))))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=100)))
model.add(Dropout(0.2))
model.add(Dense(1))

# Custom metric: Root Mean Squared Error
rmse = RootMeanSquaredError(name='rmse')

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[rmse])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Make predictions on the test data
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

# Calculate the inverse transform of 'y_test'
actual_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

# Print the actual stock prices and their corresponding predictions
for i in range(len(predictions)):
    print(f"Actual price: {actual_prices[i][0]:.2f} - Predicted price: {predictions[i][0]:.2f}")


[*********************100%%**********************]  1 of 1 completed
Epoch 1/100
10/10 [==============================] - 11s 488ms/step - loss: 0.1107 - rmse: 0.3327 - val_loss: 0.0290 - val_rmse: 0.1703
Epoch 2/100
10/10 [==============================] - 4s 443ms/step - loss: 0.0173 - rmse: 0.1316 - val_loss: 0.0081 - val_rmse: 0.0901
Epoch 3/100
10/10 [==============================] - 5s 468ms/step - loss: 0.0083 - rmse: 0.0914 - val_loss: 0.0086 - val_rmse: 0.0929
Epoch 4/100
10/10 [==============================] - 5s 455ms/step - loss: 0.0064 - rmse: 0.0800 - val_loss: 0.0045 - val_rmse: 0.0674
Epoch 5/100
10/10 [==============================] - 5s 480ms/step - loss: 0.0056 - rmse: 0.0748 - val_loss: 0.0050 - val_rmse: 0.0711
Epoch 6/100
10/10 [==============================] - 5s 517ms/step - loss: 0.0057 - rmse: 0.0752 - val_loss: 0.0041 - val_rmse: 0.0638
Epoch 7/100
10/10 [==============================] - 5s 552ms/step - loss: 0.0053 - rmse: 0.0726 - val_loss: 0.0045 - va

Epoch 61/100
10/10 [==============================] - 5s 500ms/step - loss: 0.0032 - rmse: 0.0567 - val_loss: 0.0025 - val_rmse: 0.0498
Epoch 62/100
10/10 [==============================] - 5s 492ms/step - loss: 0.0028 - rmse: 0.0527 - val_loss: 0.0022 - val_rmse: 0.0464
Epoch 63/100
10/10 [==============================] - 5s 493ms/step - loss: 0.0030 - rmse: 0.0547 - val_loss: 0.0022 - val_rmse: 0.0471
Epoch 64/100
10/10 [==============================] - 5s 500ms/step - loss: 0.0030 - rmse: 0.0551 - val_loss: 0.0019 - val_rmse: 0.0438
Epoch 65/100
10/10 [==============================] - 5s 498ms/step - loss: 0.0036 - rmse: 0.0600 - val_loss: 0.0019 - val_rmse: 0.0437
Epoch 66/100
10/10 [==============================] - 5s 497ms/step - loss: 0.0033 - rmse: 0.0576 - val_loss: 0.0020 - val_rmse: 0.0442
Epoch 67/100
10/10 [==============================] - 5s 503ms/step - loss: 0.0036 - rmse: 0.0602 - val_loss: 0.0022 - val_rmse: 0.0469
Epoch 68/100
10/10 [============================